In [1]:
import pyspark

DadolosacontecimientosenUSA,deseamosobtenerdatosquenosdenmayorinformaciónsobrelasmuertesdepersonasderazanegra por parte de oficiales de policía.Paraello,tenemosuncsvconinformaciónsobrelasmuertesporparte de oficiales de policía en USA desde 2015 hasta 2017:
(name, date, race, city, state)
Y otro csv con información sobre el porcentaje de cada raza en las ciudades de USA:
(state,city,share_white,share_black,share_native_american,share_asian,share_hispanic)

Se pide:

a)Obtener el estado con mayor porcentaje de muertes de personas de raza negra teniendo en cuenta la cantidad total de muertes por parte de oficiales en ese estado. 

In [25]:
# (name, date, race, city, state)
muertes = [
    ('Name1', 'date1', 'black', 'city1', 'state1'),
    ('Name2', 'date2', 'black', 'city1', 'state1'),
    ('Name3', 'date3', 'black', 'city1', 'state1'),
    ('Name4', 'date4', 'black', 'city1', 'state1'),
    ('Name5', 'date1', 'black', 'city2', 'state1'),
    ('Name6', 'date2', 'hispanic', 'city3', 'state2'),
    ('Name7', 'date3', 'black', 'city31', 'state2'),
    ('Name8', 'date4', 'hispanic', 'city38', 'state1'),
    ('Name9', 'date5', 'black', 'city49', 'state3'),
    ('Name10', 'date6', 'hispanic', 'city43', 'state4'),
    ('Name11', 'date1', 'black', 'city11', 'state5'),
    ('Name12', 'date2', 'black', 'city12', 'state5'),
    ('Name13', 'date3', 'asian', 'city12', 'state5'),
    ('Name14', 'date4', 'white', 'city13', 'state5'),
    ('Name15', 'date1', 'white', 'city20', 'state6'),
    ('Name16', 'date2', 'hispanic', 'city33', 'state7'),
    ('Name17', 'date3', 'black', 'city30', 'state8'),
    ('Name18', 'date4', 'black', 'city31', 'state8'),
    ('Name19', 'date5', 'black', 'city44', 'state9'),
    ('Name20', 'date6', 'black', 'city49', 'state10')
]

#(state, city, share_white, share_black, share_native_american, share_asian, share_hispanic)
porcentajeRacialPorCiudad = [
    ('state1', 'city1', 30, 40, 20, 5, 5),
    ('state1', 'city2', 5, 20, 40, 5, 5),
    ('state1', 'city28', 20, 5, 3, 2, 70),
    ('state2', 'city3', 5, 30, 40, 20, 5),
    ('state2', 'city31', 3, 2, 30, 40, 25),
    ('state3', 'city49', 30, 20, 2, 3, 45),
    ('state4', 'city43', 5, 20, 40, 5, 30),
    ('state5', 'city11', 2, 2, 6, 20, 70),
    ('state5', 'city12', 2, 5, 3, 40, 50),
    ('state5', 'city13', 20, 5, 3, 2, 70),
    ('state6', 'city20', 5, 20, 40, 5, 30),
    ('state7', 'city33', 40, 5, 2, 3, 50),
    ('state8', 'city30', 50, 40, 1, 1, 8),
    ('state8', 'city31', 2, 2, 6, 20, 70),
    ('state9', 'city44', 2, 5, 3, 40, 50),
    ('state10', 'city49', 40, 5, 2, 3, 50)
] 

In [26]:
muertesRdd = sc.parallelize(muertes)
ciudadRazasRdd = sc.parallelize(porcentajeRacialPorCiudad)

In [12]:
# mapeo tal que quede el estado como clave y como valores: 2 contadores (uno para las muertes de raza negra y 
# otro para las muertes totales) y la raza

#def sumDeaths(x,y):
#    if ((x[2] == 'black') & (y[2]=='black')):
#        return (x[0]+y[0],x[1]+y[1],x[2])

#muertes1 = muertesRdd.map(lambda x: (x[4], (1,1, x[2])))

#muertes1.collect()

In [15]:
# armo un rdd con las muertes de raza negra y los agrupo por estado para contarlos, para eso agrego un contador
# en el ultimo valor del map y luego sumo en el reduceByKey
blackDeaths = muertesRdd.filter(lambda x: x[2] == 'black')\
.map(lambda x: (x[4], 1))\
.reduceByKey(lambda x,y: x+y)

blackDeaths.collect()

[('state10', 1),
 ('state8', 2),
 ('state9', 1),
 ('state5', 2),
 ('state1', 5),
 ('state2', 1),
 ('state3', 1)]

In [18]:
# sobre el rdd original hago un map asignando al estado como clave y un contador para obtener la cantidad total
# de muertes
deaths = muertesRdd.map(lambda x: (x[4], 1))\
.reduceByKey(lambda x,y: x+y)

deaths.collect()

[('state4', 1),
 ('state10', 1),
 ('state6', 1),
 ('state7', 1),
 ('state8', 2),
 ('state9', 1),
 ('state5', 4),
 ('state1', 6),
 ('state2', 2),
 ('state3', 1)]

In [19]:
# hago un right join por estado y luego un map para poder calcular el porcentaje de muertes de raza negra por estado
comparativa = blackDeaths.rightOuterJoin(deaths)
comparativa.collect()

[('state10', (1, 1)),
 ('state4', (None, 1)),
 ('state6', (None, 1)),
 ('state8', (2, 2)),
 ('state9', (1, 1)),
 ('state7', (None, 1)),
 ('state5', (2, 4)),
 ('state1', (5, 6)),
 ('state2', (1, 2)),
 ('state3', (1, 1))]

In [32]:
# a la izquierda quedo la cantidad de muertes personas de raza negra (si es None reemplazo con 0) y  a la derecha
# la cantidad total de muertes [por estado]

porcentaje = comparativa.map(lambda x: (x[0], 0) if (x[1][0] is None) else (x[0], x[1][0]/x[1][1]*100))
porcentaje.collect()

[('state10', 100.0),
 ('state4', 0),
 ('state6', 0),
 ('state8', 100.0),
 ('state9', 100.0),
 ('state7', 0),
 ('state5', 50.0),
 ('state1', 83.33333333333334),
 ('state2', 50.0),
 ('state3', 100.0)]

In [24]:
# para obtener el estado con mayor porcentaje hago takeOrdered

porcentaje.takeOrdered(1, lambda x: -x[1])

[('state10', 1.0)]

b ) Obtener los 10 estados con mayor diferencia entre el porcentaje de muertes y el porcentaje de gente de raza negra en ese estado. Para ello, considerar el porcentaje de raza de un estado como el promedio de los valores de sus ciudades. Por ejemplo si en Texas el porcentaje de muertes de personas de raza negra por parte de la policía es del 36% y el promedio de share_black para Texas es 24% la diferencia es 0.12.

In [30]:
# necesito el promedio de porcentaje de raza negra por estado, para eso mapeo poniendo el estado como clave y 
# como valor pongo un contador y el porcentaje de raza negra
# luego hago reduceByKey para sumar los porcentajes y las cantidades
# y por ultimo un map para calcular el promedio
promedioRazaNegra = ciudadRazasRdd.map(lambda x: (x[0], (1, x[3])))\
.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
.map(lambda x: (x[0], x[1][1]/x[1][0]))

promedioRazaNegra.collect()

[('state4', 20.0),
 ('state10', 5.0),
 ('state6', 20.0),
 ('state7', 5.0),
 ('state8', 21.0),
 ('state9', 5.0),
 ('state5', 4.0),
 ('state1', 21.666666666666668),
 ('state2', 16.0),
 ('state3', 20.0)]

In [36]:
# hago un join con el rdd que calcule previamente con los porcentajes de muertes de personas de raza negra
# para comparar la diferencia porcentual
# luego hago un map para evaluar la diferencia porcentual
# finalmente me quedo con los diez con mayor diferencia

comparacionPorcentual = promedioRazaNegra.join(porcentaje)\
.map(lambda x: (x[0], abs(x[1][0]-x[1][1])))

comparacionPorcentual.collect()

[('state4', 20.0),
 ('state10', 95.0),
 ('state6', 20.0),
 ('state7', 5.0),
 ('state8', 79.0),
 ('state9', 95.0),
 ('state5', 46.0),
 ('state3', 80.0),
 ('state1', 61.66666666666667),
 ('state2', 34.0)]

In [37]:
comparacionPorcentual.takeOrdered(10, lambda x: -x[1])

[('state10', 95.0),
 ('state9', 95.0),
 ('state3', 80.0),
 ('state8', 79.0),
 ('state1', 61.66666666666667),
 ('state5', 46.0),
 ('state2', 34.0),
 ('state4', 20.0),
 ('state6', 20.0),
 ('state7', 5.0)]